# Phase 2C: 원본 소스 인덱싱 노트북

이 노트북은 수집된 원본 소스 데이터를 청킹하고 벡터 DB에 인덱싱합니다.

## 워크플로우
1. `source_collector.ipynb`에서 생성한 JSON 파일 로드
2. 소스 타입별로 적절한 청킹 전략 적용
3. 각 청크에 대해 768차원 임베딩 생성
4. Chroma 벡터 DB에 저장 (카테고리별 독립 컬렉션)

## 벡터 DB 구조
```
./chroma_db/
├── fine_dining_sources/      # 파인다이닝 원본 소스
├── mid_price_sources/         # 중저가 원본 소스
├── waiting_hotplace_sources/  # 웨이팅 핫플 원본 소스
├── fine_dining_examples/      # 기존: 요약문 예시
├── mid_price_examples/
└── waiting_hotplace_examples/
```

## 메타데이터 스키마
각 청크는 다음 메타데이터를 포함:
- `shop_seq`: 매장 시퀀스
- `shop_name`: 매장명
- `category`: 카테고리
- `source_type`: 소스 타입
- `source_url`: 출처 URL
- `chunk_index`: 청크 인덱스
- `total_chunks`: 전체 청크 수
- `indexed_at`: 인덱싱 시각

## 섹션 1: 패키지 설치

In [ ]:
%pip install google-cloud-aiplatform google-genai python-dotenv chromadb langchain langchain-text-splitters pandas

## 섹션 2: 라이브러리 Import

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import vertexai
from google import genai
from dotenv import load_dotenv
import chromadb
from datetime import datetime
from vertexai.language_models import TextEmbeddingModel
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict
import time

print("✅ 라이브러리 Import 완료")

## 섹션 3: Vertex AI 초기화

In [ ]:
load_dotenv()

PROJECT_ID = "wad-dw"
LOCATION = "us-central1"

vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

print(f"✅ Vertex AI 초기화 완료: {PROJECT_ID} ({LOCATION})")

## 섹션 4: Chroma 벡터 DB 초기화

In [ ]:
# Chroma 클라이언트 생성 (공통 벡터 DB 사용)
# 경로: knowledge_base/../chroma_db = shop_summary/chroma_db
try:
    chroma_client = chromadb.PersistentClient(path="../chroma_db")
    
    # 카테고리별 소스 컬렉션 생성 (기존 example 컬렉션과 분리)
    fine_dining_sources = chroma_client.get_or_create_collection(
        name="fine_dining_sources",
        metadata={"hnsw:space": "cosine", "description": "파인다이닝 원본 소스"}
    )
    
    mid_price_sources = chroma_client.get_or_create_collection(
        name="mid_price_sources",
        metadata={"hnsw:space": "cosine", "description": "중저가 원본 소스"}
    )
    
    waiting_hotplace_sources = chroma_client.get_or_create_collection(
        name="waiting_hotplace_sources",
        metadata={"hnsw:space": "cosine", "description": "웨이팅 핫플 원본 소스"}
    )
    
    # 컬렉션 매핑
    SOURCE_COLLECTIONS = {
        "fine_dining": fine_dining_sources,
        "mid_price": mid_price_sources,
        "waiting_hotplace": waiting_hotplace_sources
    }
    
    print("✅ Chroma 벡터 DB 초기화 완료 (공통 벡터 DB)")
    print(f"   - 저장 위치: ../chroma_db (shop_summary/chroma_db)")
    print(f"\n   ⚠️ 최초 1회 실행 후, 모든 카테고리 노트북에서 공유됩니다!")
    print(f"\n   카테고리별 컬렉션:")
    for category, collection in SOURCE_COLLECTIONS.items():
        print(f"   - {category}_sources: {collection.count()}개 청크")
    
except Exception as e:
    print(f"❌ Chroma 초기화 실패: {e}")
    chroma_client = None
    SOURCE_COLLECTIONS = {}

## 섹션 5: 청킹 및 임베딩 함수 정의

In [ ]:
# ========================================
# 청킹 설정 (source_chunking_experiment.ipynb 결과 기반)
# ========================================

CHUNKING_CONFIG = {
    "michelin_review": {
        "method": "document",
        "chunk_size": None,
        "chunk_overlap": 0
    },
    "blueribbon_review": {
        "method": "document",
        "chunk_size": None,
        "chunk_overlap": 0
    },
    "chef_interview": {
        "method": "recursive",
        "chunk_size": 800,
        "chunk_overlap": 200
    },
    "course_description": {
        "method": "semantic",
        "chunk_size": 512,
        "chunk_overlap": 50
    },
    "brand_philosophy": {
        "method": "recursive",
        "chunk_size": 600,
        "chunk_overlap": 100
    },
    "space_ambiance": {
        "method": "recursive",
        "chunk_size": 512,
        "chunk_overlap": 50
    },
    "tradition_technique": {
        "method": "recursive",
        "chunk_size": 600,
        "chunk_overlap": 100
    }
}


def chunk_text(text: str, source_type: str) -> List[str]:
    """
    소스 타입에 맞는 청킹 전략 적용
    """
    config = CHUNKING_CONFIG.get(source_type, CHUNKING_CONFIG["brand_philosophy"])
    
    if config["method"] == "document":
        # 분할 없음
        return [text]
    
    elif config["method"] == "recursive":
        # LangChain RecursiveCharacterTextSplitter
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=config["chunk_size"],
            chunk_overlap=config["chunk_overlap"],
            separators=["\n\n", "\n", ". ", "? ", "! ", " ", ""],
            length_function=len
        )
        return splitter.split_text(text)
    
    elif config["method"] == "semantic":
        # 간단한 문장 단위 분할
        sentences = text.replace(". ", ".\n").replace("? ", "?\n").replace("! ", "!\n").split("\n")
        
        chunks = []
        current_chunk = []
        current_length = 0
        chunk_size = config["chunk_size"]
        chunk_overlap = config["chunk_overlap"]
        
        for sent in sentences:
            sent = sent.strip()
            if not sent:
                continue
            
            if current_length + len(sent) > chunk_size and current_chunk:
                chunks.append(" ".join(current_chunk))
                
                if chunk_overlap > 0 and current_chunk:
                    overlap_text = current_chunk[-1]
                    current_chunk = [overlap_text, sent]
                    current_length = len(overlap_text) + len(sent)
                else:
                    current_chunk = [sent]
                    current_length = len(sent)
            else:
                current_chunk.append(sent)
                current_length += len(sent)
        
        if current_chunk:
            chunks.append(" ".join(current_chunk))
        
        return chunks
    
    return [text]


def generate_embedding(text: str, model="text-embedding-004") -> List[float]:
    """텍스트를 768차원 벡터로 변환"""
    try:
        embedding_model = TextEmbeddingModel.from_pretrained(model)
        embeddings = embedding_model.get_embeddings([text])
        return embeddings[0].values
    except Exception as e:
        print(f"❌ 임베딩 생성 실패: {e}")
        return None


def index_source_data(shop_data: Dict, collection):
    """
    한 매장의 소스 데이터를 청킹하고 벡터 DB에 저장
    
    Returns:
        저장된 청크 수
    """
    shop_seq = shop_data["shop_seq"]
    shop_name = shop_data["shop_name"]
    category = shop_data["category"]
    sources = shop_data["sources"]
    
    total_chunks_stored = 0
    
    for source_idx, source in enumerate(sources):
        source_type = source["source_type"]
        text = source["text"]
        url = source.get("url", "")
        
        # 청킹
        chunks = chunk_text(text, source_type)
        
        # 각 청크를 벡터 DB에 저장
        for chunk_idx, chunk in enumerate(chunks):
            # 임베딩 생성
            embedding = generate_embedding(chunk)
            
            if embedding is None:
                print(f"   ⚠️ 임베딩 생성 실패: {shop_name} - {source_type} 청크 {chunk_idx}")
                continue
            
            # 고유 ID 생성
            doc_id = f"{category}_{shop_seq}_{source_idx}_{chunk_idx}_{datetime.utcnow().strftime('%Y%m%d%H%M%S')}"
            
            # Chroma에 저장
            collection.add(
                documents=[chunk],
                embeddings=[embedding],
                metadatas=[{
                    "shop_seq": shop_seq,
                    "shop_name": shop_name,
                    "category": category,
                    "source_type": source_type,
                    "source_url": url,
                    "chunk_index": chunk_idx,
                    "total_chunks": len(chunks),
                    "indexed_at": datetime.utcnow().isoformat()
                }],
                ids=[doc_id]
            )
            
            total_chunks_stored += 1
            
            # API Rate Limit 방지
            time.sleep(0.1)
    
    return total_chunks_stored


print("✅ 청킹 및 임베딩 함수 정의 완료")
print("   - chunk_text(): 소스 타입별 청킹")
print("   - generate_embedding(): 768차원 임베딩 생성")
print("   - index_source_data(): 벡터 DB 인덱싱")

## 섹션 6: 수집 데이터 로드

`source_collector.ipynb`에서 생성한 JSON 파일을 로드합니다.

In [ ]:
# ========================================
# JSON 파일 로드
# ========================================

# JSON 파일 경로 입력
JSON_FILE = "collected_sources_20250111_120000.json"  # 실제 파일명으로 변경하세요

try:
    with open(JSON_FILE, 'r', encoding='utf-8') as f:
        collected_data = json.load(f)
    
    print(f"✅ 데이터 로드 완료: {JSON_FILE}")
    print(f"   - 총 매장 수: {len(collected_data)}개")
    
    # 카테고리별 통계
    category_counts = {}
    for shop in collected_data:
        cat = shop["category"]
        category_counts[cat] = category_counts.get(cat, 0) + 1
    
    print("\n   카테고리별:")
    for cat, count in category_counts.items():
        print(f"   - {cat}: {count}개")
    
    # 총 소스 수
    total_sources = sum(len(shop["sources"]) for shop in collected_data)
    print(f"\n   - 총 소스 수: {total_sources}개")
    
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {JSON_FILE}")
    print("   source_collector.ipynb를 먼저 실행하여 데이터를 수집하세요.")
    collected_data = []
except json.JSONDecodeError as e:
    print(f"❌ JSON 파싱 오류: {e}")
    collected_data = []

## 섹션 7: 벡터 DB 인덱싱 실행

⚠️ 주의: 이 섹션은 Vertex AI API를 호출하여 임베딩을 생성합니다. 시간이 오래 걸릴 수 있습니다.

In [ ]:
# ========================================
# 벡터 DB 인덱싱
# ========================================

if not collected_data:
    print("⚠️ 인덱싱할 데이터가 없습니다. 섹션 6을 먼저 실행하세요.")
else:
    print("🚀 벡터 DB 인덱싱 시작\n")
    print("=" * 80)
    print(f"\n예상 시간: 약 {len(collected_data) * 2}초 (매장당 약 2초)\n")
    
    indexing_results = []
    start_time = time.time()
    
    for idx, shop_data in enumerate(collected_data, 1):
        shop_name = shop_data["shop_name"]
        category = shop_data["category"]
        num_sources = len(shop_data["sources"])
        
        print(f"[{idx}/{len(collected_data)}] {shop_name} ({category})")
        print(f"   소스 {num_sources}개 처리 중...")
        
        # 카테고리에 맞는 컬렉션 선택
        collection = SOURCE_COLLECTIONS.get(category)
        
        if not collection:
            print(f"   ❌ 유효하지 않은 카테고리: {category}")
            indexing_results.append({
                "shop_name": shop_name,
                "category": category,
                "status": "error",
                "error": "Invalid category"
            })
            continue
        
        try:
            # 인덱싱 실행
            shop_start = time.time()
            chunks_stored = index_source_data(shop_data, collection)
            shop_elapsed = time.time() - shop_start
            
            print(f"   ✅ 완료: {chunks_stored}개 청크 저장 ({shop_elapsed:.1f}초)\n")
            
            indexing_results.append({
                "shop_name": shop_name,
                "category": category,
                "status": "success",
                "chunks_stored": chunks_stored,
                "elapsed_time": shop_elapsed
            })
            
        except Exception as e:
            print(f"   ❌ 오류: {e}\n")
            
            indexing_results.append({
                "shop_name": shop_name,
                "category": category,
                "status": "error",
                "error": str(e)
            })
    
    total_elapsed = time.time() - start_time
    
    print("=" * 80)
    print(f"\n✅ 인덱싱 완료 (총 {total_elapsed:.1f}초)\n")
    
    # 통계
    success_count = sum(1 for r in indexing_results if r["status"] == "success")
    error_count = len(indexing_results) - success_count
    total_chunks = sum(r.get("chunks_stored", 0) for r in indexing_results)
    
    print(f"📊 통계:")
    print(f"   - 성공: {success_count}개 매장")
    if error_count > 0:
        print(f"   - 실패: {error_count}개 매장")
    print(f"   - 총 청크: {total_chunks}개")
    print(f"   - 평균 처리 시간: {total_elapsed / len(collected_data):.1f}초/매장")
    
    # 카테고리별 컬렉션 크기
    print(f"\n   카테고리별 컬렉션 크기:")
    for category, collection in SOURCE_COLLECTIONS.items():
        print(f"   - {category}_sources: {collection.count()}개 청크")

## 섹션 8: 결과 저장

In [ ]:
# ========================================
# 인덱싱 결과 저장
# ========================================

if indexing_results:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # CSV 저장
    df_results = pd.DataFrame(indexing_results)
    csv_filename = f"indexing_results_{timestamp}.csv"
    df_results.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    
    print(f"💾 결과 저장: {csv_filename}")
    print(f"\n" + "=" * 80)
    print(df_results.to_string(index=False))
else:
    print("⚠️ 저장할 결과가 없습니다.")

## 섹션 9: 검색 테스트 (선택)

인덱싱된 데이터가 제대로 검색되는지 테스트합니다.

In [ ]:
# ========================================
# 검색 테스트
# ========================================

# 테스트 쿼리
TEST_QUERY = "셰프의 조리 철학과 제철 식재료 활용 방법"

print(f"🔍 검색 테스트\n")
print(f"쿼리: {TEST_QUERY}\n")
print("=" * 80)

# 쿼리 임베딩 생성
query_embedding = generate_embedding(TEST_QUERY)

if query_embedding:
    # 각 카테고리별로 검색
    for category, collection in SOURCE_COLLECTIONS.items():
        if collection.count() == 0:
            print(f"\n📂 {category}: 저장된 청크 없음")
            continue
        
        print(f"\n📂 {category}")
        print("─" * 80)
        
        try:
            results = collection.query(
                query_embeddings=[query_embedding],
                n_results=min(3, collection.count()),
                include=["documents", "metadatas", "distances"]
            )
            
            if results['ids'] and len(results['ids'][0]) > 0:
                for i in range(len(results['ids'][0])):
                    metadata = results['metadatas'][0][i]
                    document = results['documents'][0][i]
                    distance = results['distances'][0][i]
                    similarity = 1.0 - (distance / 2.0)
                    
                    print(f"\n   [{i+1}] {metadata['shop_name']} - {metadata['source_type']}")
                    print(f"      유사도: {similarity:.3f}")
                    print(f"      청크: {metadata['chunk_index'] + 1}/{metadata['total_chunks']}")
                    
                    preview = document[:150] + "..." if len(document) > 150 else document
                    print(f"      내용: {preview}")
            else:
                print("   검색 결과 없음")
                
        except Exception as e:
            print(f"   ❌ 검색 오류: {e}")
    
    print("\n" + "=" * 80)
else:
    print("❌ 쿼리 임베딩 생성 실패")

## 다음 단계

이제 원본 소스가 벡터 DB에 인덱싱되었습니다!

### 다음 노트북
1. ✅ **source_chunking_experiment.ipynb** - 청킹 전략 실험
2. ✅ **source_collector.ipynb** - 원본 데이터 수집
3. ✅ **source_indexer.ipynb** (현재) - 벡터 DB 인덱싱
4. ⏭️ **main_rag_source.ipynb** - Source-based RAG (원본 소스만 사용)
5. ⏭️ **main_rag_hybrid.ipynb** - Hybrid RAG (원본 소스 + 요약문 예시)

### 현재 벡터 DB 구조
```
./chroma_db/
├── fine_dining_sources/      ← 새로 추가 (원본 소스)
├── mid_price_sources/         ← 새로 추가
├── waiting_hotplace_sources/  ← 새로 추가
├── fine_dining_examples/      (기존: 요약문 예시)
├── mid_price_examples/
└── waiting_hotplace_examples/
```